<a href="https://colab.research.google.com/github/JangAyeon/ToBigs/blob/master/%5B4%5DSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### TOBIG'S 14기 정규세션 4주차 SVM 
### ASSIGNMENT1. Multiclass SVM 구현

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [ ]:
scal = StandardScaler() #scaling
X = scal.fit_transform(X)

In [ ]:
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [ ]:
# One VS Rest 구현 위한 class 구분 머신 생성

svm_1 = SVC(kernel ='rbf', C = 5, gamma = 5) #class: 0 or not 구분할 머신
svm_2 = SVC(kernel ='rbf', C = 5, gamma = 5) #class: 1 or not 구분할 머신
svm_3 = SVC(kernel ='rbf', C = 5, gamma = 5) #class: 2 or not 구분할 머신

In [ ]:
 #test/train 데이터로 분리
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [ ]:
#one hot encoding

y_train = pd.get_dummies(y_train) 

In [ ]:
#class: 0 or not 구분
svm_1.fit(X_train,y_train.iloc[:,0])
#class: 1 or not 구분
svm_2.fit(X_train,y_train.iloc[:,1]) 
#class: 2 or not 구분
svm_3.fit(X_train,y_train.iloc[:,2]) 


#svm_1을 이용해 데이터의 class가 0인지 아닌지를 예측
print(svm_1.predict(X_test)) 

#decision_function : hyperplane과의 거리를 구함
print(svm_1.decision_function(X_test))

[0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[-1.12470845 -0.86326953 -0.65281099 -0.50248821 -0.76284323 -0.87465573
  1.07709345 -0.99281647  0.47441336 -0.99842743 -0.83989348  0.15633457
  0.32871788 -0.97965464 -0.72385083 -0.92638376  1.28322481 -0.56240455
 -0.72719892 -0.99509775  0.43166724 -0.96451557 -0.82991366 -1.03020581
 -0.75166835  1.13461335  0.39943974 -1.04194106 -0.93376548 -1.06133798]


* 배열 원소의 부호 판별 함수 : `np.sign(x)`  ☞ 1 (positive), 0(zero), -1(negative)

In [ ]:
for i in range(len(X_test)): #전체를 돌면서 
    if (np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i])) and (np.sign(svm_2.decision_function(X_test)[i]) == np.sign(svm_3.decision_function(X_test)[i])):
      #decision function을 이용해 data~hyperplane의 거리 구함
      #그 거리의 부호가 같은지 확인함 => 같은 경우 모두 동점인 경우가 생긴 것
        print(i) #동점인 경우의 인덱스 출력

3
17
18


#One vs Rest

In [ ]:
#class 0 1 2를 구분한 머신을 모음
classifier=[]
classifier.append(svm_1)
classifier.append(svm_2)
classifier.append(svm_3)

#구분 머신을 바탕으로 진행한 voting 결과를 기록할 scoring 배열 생성
scoring=np.zeros((150,3),dtype=int)
#예측 값을 기록할 pred
pred=[]

for i in range(len(X_test)): #전체를 톨면서
  if classifier[0].predict(X_test)[i]==1: #class 0인 경우 
    scoring[i][0]+=1 #0 class 점수로 +1
    scoring[i][1]-=1 #1 class 점수로 -1
    scoring[i][2]-=1 #2 class 점수로 -1

  elif classifier[1].predict(X_test)[i]==1: #class 1인 경우 
    scoring[i][0]-=1 #0 class 점수로 -1
    scoring[i][1]+=1 #1 class 점수로 +1
    scoring[i][2]-=1 #2 class 점수로 -1

  elif classifier[2].predict(X_test)[i]==1: #class 2인 경우
    scoring[i][0]-=1 #0 class 점수로 -1
    scoring[i][1]-=1 #1 class 점수로 -1
    scoring[i][2]+=1 #2 class 점수로 +1

  pred.append(np.argmax(scoring[i])) #가장 큰 점수를 가진 것을 예측 값으로!! 선정


  if (np.sign(classifier[0].decision_function(X_test)[i]) == np.sign(classifier[1].decision_function(X_test)[i])): #투표 점수 결과가 [0] [1] [2] 모두 같은 경우(=>모두 0인 경우)
    if (np.sign(classifier[1].decision_function(X_test)[i]) == np.sign(classifier[2].decision_function(X_test)[i])):
      pred[i] = np.argmax([classifier[0].decision_function(X_test)[i], classifier[1].decision_function(X_test)[i], classifier[2].decision_function(X_test)[i]])  #decision function 값이 가장 큰 값을 예측값으로

  #[0][1] / [1][2] / [0] [2] 의 투표 점수 결과가 모두 양수이고 같은 경우
  #decision function 값이 가장 큰 값을 예측값으로
  elif (scoring[i][0] > 0 and scoring[i][1] > 0):
    if  (scoring[i][0] == scoring[i][1]):
      pred[i] = np.argmax([classifier[0].decision_function(X_test)[i], classifier[1].decision_function(X_test)[i]])
  elif (scoring[i][0] > 0 and scoring[i][2]) > 0:
    if  (scoring[i][0] == scoring[i][1]):  
      pred[i] = np.argmax([classifier[0].decision_function(X_test)[i], classifier[2].decision_function(X_test)[i]])
  elif  (scoring[i][1] > 0 and scoring[i][2] > 0): 
    if  (scoring[i][1] == scoring[i][2]):
      pred[i] = np.argmax([classifier[1].decision_function(X_test)[i], classifier[2].decision_function(X_test)[i]])

   #투표를 진행하며 0,1,2로 되어있던 값을 class값으로 변환
y_pred = pd.DataFrame(pred).replace({0:'setosa', 1:'versicolor', 2:'virginica'})
print("Accuacy : ", accuracy_score(y_test, y_pred))


Accuacy :  0.8666666666666667


In [ ]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

sc = StandardScaler() 
X_train_2 = sc.fit_transform(X_train_2)
X_test_2 = sc.transform(X_test_2)

svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

accuracy_score(y_test_2,y_pred)

0.8666666666666667